# Importing Packages and establishing connections

In [34]:
import pandas as pd
import json
import requests
from socketIO_client import SocketIO

In [35]:
TRADING_API_ENDPOINT = 'https://api-demo.fxcm.com:443'
WEBSOCKET_PORT = 443
ACCESS_TOKEN = "1314974d8bc306a71afe37eade70dadcf03e9093"

In [36]:
def when_connected():
    print('WebSocket Connected: ' + socketIO._engineIO_session.id)
    
def when_closed():
    print('WebSocket Disconnected')
    
socketIO = SocketIO(TRADING_API_ENDPOINT, WEBSOCKET_PORT, params = {'access_token': ACCESS_TOKEN})

socketIO.on('connect', when_connected)
socketIO.on('disconnect', when_closed)

bearer_token = "Bearer " + socketIO._engineIO_session.id + ACCESS_TOKEN
print(bearer_token)

Bearer 6rH3XZgWKsKVk74bABM21314974d8bc306a71afe37eade70dadcf03e9093


# Pulling Historical Data (Pricing)


In [37]:
method = '/candles/1/H1'

hist_response = requests.get(TRADING_API_ENDPOINT + method,
                            headers = { 
                                'User-Agent': 'request',
                                'Authorization': bearer_token,
                                'Accept': 'application/json',
                                'Content-Type': 'application/x-www-form-urlencoded'
                            },
                            params = {
                                'num': 1000, #number of ticks
                                'from': 1546300800, # epoch for 1 Jan 2019
                                'to': 1570194000
                            })

In [38]:
print(hist_response)
print (hist_response.json())

<Response [200]>
{'response': {'executed': True, 'error': ''}, 'instrument_id': '1', 'period_id': 'H1', 'candles': [[1546405200, 1.14442, 1.14774, 1.14782, 1.14555, 1.14459, 1.14788, 1.14797, 1.14571, 5376], [1546408800, 1.14774, 1.14846, 1.14961, 1.14763, 1.14788, 1.14862, 1.14975, 1.14778, 11891], [1546412400, 1.14846, 1.14759, 1.14925, 1.14731, 1.14862, 1.14775, 1.14939, 1.14744, 15746], [1546416000, 1.14759, 1.14473, 1.14774, 1.1428, 1.14775, 1.14489, 1.14788, 1.14295, 33857], [1546419600, 1.14473, 1.14334, 1.14491, 1.14214, 1.14489, 1.14348, 1.14505, 1.14228, 27032], [1546423200, 1.14334, 1.14363, 1.14411, 1.14313, 1.14348, 1.14378, 1.14425, 1.14327, 17815], [1546426800, 1.14363, 1.14331, 1.14383, 1.14239, 1.14378, 1.14345, 1.14398, 1.14253, 15980], [1546430400, 1.14331, 1.14043, 1.14334, 1.1397, 1.14345, 1.14058, 1.14348, 1.13985, 19976], [1546434000, 1.14043, 1.13709, 1.14049, 1.13666, 1.14058, 1.13724, 1.14064, 1.13681, 33837], [1546437600, 1.13709, 1.13601, 1.13747, 1.13572, 1

In [39]:
if hist_response.status_code == 200:
    print("Data retrieved.....")
    
    hist_data = hist_response.json()
    candle_data = hist_data['candles']
    
    data = pd.DataFrame(candle_data)
    data.columns = ["time", "bidopen", "bidclose", "bidhigh", "bidlow", "askopen", "askclose", "askhigh", "asklow", "TickQty"]
    data["time"] = pd.to_datetime(data['time'], unit = 's')
    print(data)
else:
    print(hist_response)

Data retrieved.....
                    time  bidopen  bidclose  bidhigh   bidlow  askopen  \
0    2019-01-02 05:00:00  1.14442   1.14774  1.14782  1.14555  1.14459   
1    2019-01-02 06:00:00  1.14774   1.14846  1.14961  1.14763  1.14788   
2    2019-01-02 07:00:00  1.14846   1.14759  1.14925  1.14731  1.14862   
3    2019-01-02 08:00:00  1.14759   1.14473  1.14774  1.14280  1.14775   
4    2019-01-02 09:00:00  1.14473   1.14334  1.14491  1.14214  1.14489   
...                  ...      ...       ...      ...      ...      ...   
4821 2019-10-04 07:00:00  1.09654   1.09712  1.09742  1.09636  1.09666   
4822 2019-10-04 08:00:00  1.09712   1.09731  1.09756  1.09677  1.09725   
4823 2019-10-04 09:00:00  1.09731   1.09780  1.09823  1.09722  1.09742   
4824 2019-10-04 10:00:00  1.09779   1.09882  1.09891  1.09756  1.09791   
4825 2019-10-04 11:00:00  1.09882   1.09807  1.09882  1.09784  1.09894   

      askclose  askhigh   asklow  TickQty  
0      1.14788  1.14797  1.14571     5376  
1  

# Streaming Real time Forex Prices

In [40]:
method = '/subscribe'

sub_response = requests.post(TRADING_API_ENDPOINT + method,
                            headers = { 
                                'User-Agent': 'request',
                                'Authorization': bearer_token,
                                'Accept': 'application/json',
                                'Content-Type': 'application/x-www-form-urlencoded'
                            },
                             data = {
                                 'pairs': 'EUR/USD'
                             })

print(sub_response)
print(sub_response.json())

def price_updates(message):
    response = json.loads(message)
    print(response)
    
if  sub_response.status_code == 200:
    socketIO.on('EUR/USD', price_updates)
    socketIO.wait()


<Response [200]>
{'response': {'executed': True, 'error': ''}, 'pairs': [{'Updated': 1570193944975, 'Rates': [1.09641, 1.09655, 1.10009, 1.09565, 1.09641, 1.09655], 'Symbol': 'EUR/USD'}]}
WebSocket Connected: 6rH3XZgWKsKVk74bABM2
{'Updated': 1570193947253, 'Rates': [1.09649, 1.0966099999999999, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193948267, 'Rates': [1.09651, 1.0966399999999998, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193949272, 'Rates': [1.09656, 1.09669, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193950272, 'Rates': [1.09653, 1.09666, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193951246, 'Rates': [1.09651, 1.0966399999999998, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193951986, 'Rates': [1.0965, 1.09663, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD'}
{'Updated': 1570193952965, 'Rates': [1.0965, 1.09663, 1.1000899999999998, 1.09565], 'Symbol': 'EUR/USD

KeyboardInterrupt: 